In [1]:
# sourcery skip: avoid-global-variables

import pandas as pd
import os

import skombo
from tabulate import tabulate
import matplotlib
from pandas.io.formats.style import Styler
from skombo.fd_ops import get_fd,clean_fd,FdBotCsvManager,FrameData

get_fd()
FD = clean_fd()

LOG = skombo.LOG


test_combo_csv_path = os.path.join(
    skombo.ABS_PATH,
    (skombo.CHARS.AN.lower() + skombo.TEST_COMBOS_SUFFIX),
)
pd.options.display.max_rows = None # type: ignore
pd.options.display.max_columns = None # type: ignore
pd.options.display.width = None # type: ignore
pd.options.display.max_colwidth = None # type: ignore
pd.options.display.precision = 2 # type: ignore
pd.options.display.float_format = "{:,.2f}".format # type: ignore

Constants initialized
ABS_PATH: C:\Users\micah\repos\skugs\src\skombo
MODULE_NAME: skombo
Logger initialized


In [2]:
from skombo import COLS
from skombo import CHARS
from skombo.utils import filter_dict
#FD.loc[CHARS.AN, columns_to_keep].style.pipe(make_look_good)  # type: ignore

def make_look_good(styler: Styler):
    styler.set_caption("Frame Data")
    styler.background_gradient(cmap="Blues")
    styler.set_properties(**{"border": "1px solid black", "border-collapse": "collapse"})
    return styler

FD.style.pipe(make_look_good)  



